In [1]:
import numpy as np
from .extractor.state import XYMStateMap
import networkx as nx 

import plotly.graph_objects as go
import dash
import jupyter_dash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

cyto.load_extra_layouts()


In [2]:
state_map = XYMStateMap.read_msgpack('../extractor/output/state_map.msgpack')

In [3]:
MIN_HARVESTER = 500000 # smallest harvester you want to see rendered in terms of balance
MIN_NODE = 10000000 # smallest node you want to see rendered in terms of total balance
SCALE = 250 # starting scale to get a decent layout for the fixed points
GRID_SPACE = 3.5*SCALE

t = 0
node_pos = {}
center = np.zeros(2)
cc_map = {}

In [22]:
pos = {}
node_graph = state_map.get_harvester_graph(1000)

for cc in sorted(nx.connected_components(node_graph.to_undirected()),key=len,reverse=True):

    cg = nx.subgraph(node_graph,cc).copy()

    # TODO: track old subgraph components, keep their positions fixed to minimize moving parts
    # TODO: keep track of centers and nodes, free centers when nodes are no longer present

    for node in cc:
        if cg.nodes()[node]['type'] == 'node':
            cg_center = node
            if node not in node_pos.keys():
                j = np.sqrt(t).round()
                k = np.abs(j**2 - t) - j 
                center[0] = ((k + j ** 2 - t - (j % 2)) * 0.5 * (-1) ** j) * GRID_SPACE
                center[1] = ((-k + j ** 2 - t - (j % 2)) * 0.5 * (-1) ** j) * GRID_SPACE
                node_pos[node] = center.copy()
                t += 1
            pos[node] = center.copy()

    if len(cc) > 1:
        pos[cg_center] = node_pos[cg_center]
        cg.remove_node(cg_center)
        cg_pos = nx.circular_layout(cg,scale=SCALE,center=node_pos[cg_center])
    else:
        cg_pos = {next(iter(cc)):node_pos[cg_center]}

    pos.update(cg_pos)        

layout_pos = {}
for node in node_graph.nodes():
    layout_pos[node] = {'x':pos[node][0],'y':pos[node][1]}

# start figure generation

edge_x = []
edge_y = []
for edge in node_graph.edges():
    x0, y0 = layout_pos[edge[0]].values()
    x1, y1 = layout_pos[edge[1]].values()
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scattergl(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
node_color = []
node_size = []
node_text = []
for node in node_graph.nodes():
    attr = node_graph.nodes[node]
    node_x.append(layout_pos[node]['x'])
    node_y.append(layout_pos[node]['y'])
    node_color.append(attr['link_age'])
    node_size.append(np.log2(attr['balance']))
    node_text.append(f"id: {node} <br>balance: {attr['balance']}")

# probably going to make separate traces for nodes and for delegates . . . 

node_trace = go.Scattergl(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='BlueRed',
        reversescale=True,
        color=node_color,
        size=node_size,
        colorbar=dict(
            thickness=15,
            title='Link Age (Blocks)',
            xanchor='left',
            titleside='right'
        ),
        cmin=0,
        cmax=1000,
        line_width=2))

node_trace.marker.color = node_color
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
     layout=go.Layout(
        plot_bgcolor='white',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
        )

fig.show()



In [18]:
node_text

['id: NANCELB3O6ZOZIPMHOT55OERDCUVCJWZM63VNSI <br>balance: 15235828.824523',
 'id: NAMAHMWDKUZEUHYYS7GA62ZB2JC6DDYZKTEORTA <br>balance: 5736397.926497',
 'id: NDWEA7LK56D36YKHFRPBWND3WPYZOKE7DUIMFWQ <br>balance: 5552866.068534',
 'id: NDVRXI7QZGOI2M6FBIY5ZBBEARKLGZP2OZWEUCY <br>balance: 4463730.205316',
 'id: NBXBYYZLTO4BSRTVHJDPYI3ESF4MB4VUTIAS45Q <br>balance: 3681238.031401',
 'id: NCV5FRMHB5P2EIDEWO63FSL4WEID72HWEJKOLHI <br>balance: 3242513.555421',
 'id: NDUF24CLX7FWNTTFECIWOAIIEK7C4MLZCY53PKI <br>balance: 3015800.748361',
 'id: NCEKOSYOJX2BZLLCUXN74X63YD4LCF5DSCF734Q <br>balance: 3010342.471362',
 'id: ND3COXRB5J5JPO7K5XZPW3D7MMWJQ64KDIU7S2Y <br>balance: 2991725.14131',
 'id: ND4HYWW5NDJRZECFL4OOQ2MRL7RNBW27Y7QPQJY <br>balance: 3001567.806693',
 'id: NA2KCMY32Z4H34E5F5H2UFI4ASNXKHLY7WHFNWI <br>balance: 3001575.122572',
 'id: NCVB6RQVBI3NNFWXGXFVT74Z5DB6QFEIMSHQH6Q <br>balance: 3001569.893439',
 'id: NDRWHSSYJRYHMSEE6ALUSUXJ3WFW4ECRA5NAVJY <br>balance: 3001575.590011',
 'id: NA5VNF

In [24]:
app = jupyter_dash.JupyterDash(__name__)


app.layout = html.Div([
    dcc.Graph(
        id='harvesters-graph',
        style={'width': '100%', 'height': '95vh'},
        config={'scrollZoom': True},
        responsive=True
    ),
    dcc.Slider(
        id='height-slider',
        min=0,
        max=max([max(x['xym_balance'].keys()) for x in state_map.values() if len(x['xym_balance'])]),
        step=100,
        value=1000,
        tooltip={'always_visible':True,'placement':'bottom'}
    ),  
])


@app.callback(Output('harvesters-graph', 'figure'),
              Input('height-slider', 'value'),
              State('harvesters-graph', 'figure'))
def update_graph(height,fig):
    if height is None:
        raise PreventUpdate
        
    global t
    global node_pos
    pos = {}
    node_graph = state_map.get_harvester_graph(height) #,min_harvester_size=MIN_HARVESTER)

    for cc in sorted(nx.connected_components(node_graph.to_undirected()),key=len,reverse=True):

        cg = nx.subgraph(node_graph,cc).copy()

        # TODO: track old subgraph components, keep their positions fixed to minimize moving parts
        # TODO: keep track of centers and nodes, free centers when nodes are no longer present

        for node in cc:
            if cg.nodes()[node]['type'] == 'node':
                cg_center = node
                if node not in node_pos.keys():
                    j = np.sqrt(t).round()
                    k = np.abs(j**2 - t) - j 
                    center[0] = ((k + j ** 2 - t - (j % 2)) * 0.5 * (-1) ** j) * GRID_SPACE
                    center[1] = ((-k + j ** 2 - t - (j % 2)) * 0.5 * (-1) ** j) * GRID_SPACE
                    node_pos[node] = center.copy()
                    t += 1
                pos[node] = center.copy()
                
        if len(cc) > 1:
            pos[cg_center] = node_pos[cg_center]
            cg.remove_node(cg_center)
            cg_pos = nx.circular_layout(cg,scale=SCALE,center=node_pos[cg_center])
        else:
            cg_pos = {next(iter(cc)):node_pos[cg_center]}

        pos.update(cg_pos)        

    layout_pos = {}
    for node in node_graph.nodes():
        layout_pos[node] = {'x':pos[node][0],'y':pos[node][1]}

    # start figure generation
        
    edge_x = []
    edge_y = []
    for edge in node_graph.edges():
        x0, y0 = layout_pos[edge[0]].values()
        x1, y1 = layout_pos[edge[1]].values()
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scattergl(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    node_size = []
    node_text = []
    for node in node_graph.nodes():
        attr = node_graph.nodes[node]
        node_x.append(layout_pos[node]['x'])
        node_y.append(layout_pos[node]['y'])
        node_color.append(attr['link_age'])
        node_size.append(np.log2(attr['balance']))
        node_text.append(f"id: {node} <br>balance: {attr['balance']}")

    # probably going to make separate traces for nodes and for delegates . . . 

    node_trace = go.Scattergl(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='BlueRed',
            reversescale=True,
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Link Age (Blocks)',
                xanchor='left',
                titleside='right'
            ),
            cmin=0,
            cmax=1000,
            line_width=2))

    node_trace.marker.color = node_color
    node_trace.text = node_text

    if fig is None:
        fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                plot_bgcolor='white',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
    else:
        fig['data'] = [edge_trace, node_trace]
    
    return fig


app.run_server(mode='external', host='0.0.0.0', port=8050, threaded=True, debug=True)

Dash app running on http://0.0.0.0:8050/
